In [2]:
from pyfiles import (parametrics as par,
    complex as cplx,
    matrix as mat,
    vineyard as vin,
    input as inp,
    plot as ourplot,
    grid as grid,
    utils as utils,
    examples as ex,
    sneaky_matrix as sneaky
)
from pyfiles.examples import CameraOpt

# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2
%env RUST_BACKTRACE=1

import numpy as np
from copy import deepcopy

from typing import *

import matplotlib.pyplot as plt
%matplotlib inline

import mars

env: RUST_BACKTRACE=1


In [3]:
filename = 'input/hinge.obj'

class vineyard2(vin.vineyard):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.swaps = []

  def get_point_key(self, point: np.ndarray):
    return None
  
  def prune(self, s1: cplx.simplex, s2: cplx.simplex, old_state, new_state):
    if s1.dim() == s2.dim():
      self.swaps.append((s1, s2, old_state, new_state))
    return True

p0 = [0.3, 0.3, 0.3]
p1 = [0.6, 0.3, 0.3]


py_complex = inp.read_obj(filename)
py_vineyard = vineyard2(py_complex)
py_state0 = py_vineyard.reduce(p0)
py_state1 = py_vineyard.reduce_vine(py_state0, p1)

print('Python swaps')
print(f'#swaps = {len(py_vineyard.swaps)}')
print('   '.join([f'{a.prettyrepr()}-{b.prettyrepr()}' for (a, b, _, _) in py_vineyard.swaps]))


rs_complex = mars.read_from_obj(filename)
rs_state0 = mars.reduce_from_scratch(rs_complex, p0)
(rs_state1, swaps) = mars.vineyards_123(rs_complex, rs_state0, p1)

sdict = {0: 'v', 1: 'e', 2: 'f'}
print()
print('Rust swaps') 
print(f'#swaps = {len(swaps)}')
print('   '.join([f'{sdict[dim]}{a}-{sdict[dim]}{b}' for (dim, (a, b)) in swaps]))


Python swaps
#swaps = 1
e2-e1

Rust swaps
#swaps = 2
e3-e4   e1-e3


In [4]:
cplx = mars.read_from_obj('input/three-points.obj')

pt0 = [0.8, 0.55, 0]
pt1 = [0.8, 0.45, 0]
pt2 = [0.8, 0.35, 0]

state0 = mars.reduce_from_scratch(cplx, pt1)
(state1, swaps) = mars.vineyards_123(cplx, state0, pt2)
assert len(swaps) == 0

##############################################################
import sys

sys.stdout.flush()
print()
print('Reduce from scratch', pt0)
state0 = mars.reduce_from_scratch(cplx, pt0)
sys.stdout.flush()
print()

print('vineyards to ', pt1)
(state1, swaps1) = mars.vineyards_123(cplx, state0, pt1)
sys.stdout.flush()
assert len(swaps1) > 0, 'should have swaps'
print()

print('vineyards to ', pt2)
(state2, swaps2) = mars.vineyards_123(cplx, state1, pt2)
sys.stdout.flush()
assert len(swaps2) == 0, 'should not have swaps'
print()


Reduce from scratch [0.8, 0.55, 0]

vineyards to  [0.8, 0.45, 0]

vineyards to  [0.8, 0.35, 0]



In [5]:
print(str(state1.stacks[0].U_t))

s1 = mars.reduce_from_scratch(cplx, pt1)
print(str(s1.stacks[0].U_t))

|×  |
|×× |
|×××|

|×  |
|×× |
|× ×|

